In [20]:
from docplex.mp.model import Model
import numpy as np
from source_torch.mlca.mlca_nn import MLCA_NN
from source_torch.mlca.mlca_nn_mip import MLCA_NNMIP
from collections import OrderedDict
import logging
import source_torch.util as util
import pandas as pd

In [21]:
class MLCA_GALO_MIP:
    def __init__(self, value_model, bidder_id, presampled_algorithm='unif', presampled_n=10, L = 6000):
        self.bidder_id = bidder_id
        self.Mip = Model(name='MLCA_GALO_MIP')
        self.presampled_algorithm = presampled_algorithm
        self.presampled_n = presampled_n
        #if self.presampled_algorithm == 'gali':
        #    D_presampled = util.gali_bids_init(value_model=value_model, bidder_id=bidder_id, n=presampled_n)
        #else:
        #    D_presampled = util.unif_random_bids(value_model=value_model, bidder_id=bidder_id, n=presampled_n)
        #self.values = D_presampled[:, -1].tolist()
        
        #self.bundles = D_presampled[:, :-1].tolist()
        
        self.values = np.random.randint(1, 100, size=(self.presampled_n, 1)).tolist()
        self.bundles = np.random.randint(0, 2, size=(self.presampled_n, 10)).tolist()
        self.M = len(self.bundles[0])#number of items
        self.bidder_model = self.train()
        self.bidder_name = 'Bidder 0'
        #test = self._get_model_layer_shapes(self.bidder_model, layer_type=['dense', 'input'])
        self.L = L  # global big-M variable: see paper
        print(self._get_model_layer_shapes(self.bidder_model, layer_type=['dense', 'input']))
        #print(self._get_model_layer_shapes(self.bidder_model, layer_type=['dense', 'input']))
        self.upper_bounds_z = {self.bidder_name: np.array([self.L]*self._get_model_layer_shapes(self.bidder_model, layer_type=['dense', 'input'])).reshape(-1, 1)}
        self.upper_bounds_s = {self.bidder_name: np.array([self.L]*self._get_model_layer_shapes(self.bidder_model, layer_type=['dense', 'input'])).reshape(-1, 1)}
        self.z = {}  # MIP variable: see paper
        self.s = {}  # MIP variable: see paper
        self.y = {}  # MIP variable: see paper
        self.x_star = np.ones((self.N, self.M))*(-1)  # optimal allocation (-1=not yet solved)
        self.soltime = None  # timing
        
                

    def train(self):
        model = MLCA_NN(X_train = np.array(self.bundles), Y_train=np.array(self.values))
        model.initialize_model(model_parameters={'learning_rate': 0.001, 'architecture': [10, 10, 10], 'dropout': False, 'dropout_prob': 0.2, 'device': 'cpu', 'regularization_type': 'l2', 'regularization': 0.01})
        model.fit(epochs=10, batch_size=32)
        return model
    
    def _get_model_weights(model): # torch
        nnmodel = model.model
        weights = []
        for params in nnmodel.parameters():
            weights.append(params.detach().cpu().numpy().T)        
        return weights

    def _get_model_layer_shapes(model, layer_type=None):
        ''' return layer output shapes instead, 
            if 'input' is given as desired layer type, insert input dim at the beginning.
            assumes torch model '''
        # nnmodel = self.Models[key]
        nnmodel = model.model
        Layer_shapes = []
        for i, (name, param) in enumerate(nnmodel.named_parameters()):
            if (i==0) and ('input' in layer_type): 
                Layer_shapes.append(param.shape[1])
            if any([x in name for x in layer_type]) and ('bias' in name):
                Layer_shapes.append(param.shape[0])
        return Layer_shapes

    def _clean_weights(Wb):
        for v in range(0, len(Wb)-2, 2):
            Wb[v][abs(Wb[v]) <= 1e-8] = 0
            Wb[v+1][abs(Wb[v+1]) <= 1e-8] = 0
            zero_rows = np.where(np.logical_and((Wb[v] == 0).all(axis=0), Wb[v+1] == 0))[0]
            if len(zero_rows) > 0:
                logging.debug('Clean Weights (rows) %s', zero_rows)
                Wb[v] = np.delete(Wb[v], zero_rows, axis=1)
                Wb[v+1] = np.delete(Wb[v+1], zero_rows)
                Wb[v+2] = np.delete(Wb[v+2], zero_rows, axis=0)
        return(Wb)
    

    def optimization_step_nn(self):
        attempts = self.MIP_parameters['attempts_DNN_WDP']
        for attempt in range(1, attempts+1):
            logging.debug('Initialize MIP for GALO(NN)')
            #X = MLCA_NNMIP(DNNs, L=self.MIP_parameters['bigM'])
            if not self.MIP_parameters['mip_bounds_tightening']:
                print('MIP bounds not tightening is not implemented yet')
            elif self.MIP_parameters['mip_bounds_tightening'] == 'IA':
                #This is what usually happens
                self.tighten_bounds_IA(upper_bound_input=[1]*self.M)
                #X.print_upper_bounds(only_zeros=True)
                X.initialize_mip(verbose=False, bidder_specific_constraints=bidder_specific_constraints)
            elif self.MIP_parameters['mip_bounds_tightening'] == 'LP':
                print('MIP bounds tightening == LP is not implemented yet')
                #X.tighten_bounds_IA(upper_bound_input=[1]*self.M)
                #X.tighten_bounds_LP(upper_bound_input=[1]*self.M)
                #X.print_upper_bounds(only_zeros=True)
                #X.initialize_mip(verbose=False, bidder_specific_constraints=bidder_specific_constraints)
            try:
                logging.info('Solving MIP')
                logging.info('Attempt no: %s', attempt)
                if self.MIP_parameters['warm_start'] and self.warm_start_sol[economy_key] is not None:
                    logging.debug('Using warm start')
                    self.warm_start_sol[economy_key] = X.solve_mip(log_output=False, time_limit=self.MIP_parameters['time_limit'],
                                                                   mip_relative_gap=self.MIP_parameters['relative_gap'], integrality_tol=self.MIP_parameters['integrality_tol'],
                                                                   mip_start=docplex.mp.solution.SolveSolution(X.Mip, self.warm_start_sol[economy_key].as_dict()))
                else:
                    self.warm_start_sol[economy_key] = X.solve_mip(log_output=False, time_limit=self.MIP_parameters['time_limit'],
                                                                   mip_relative_gap=self.MIP_parameters['relative_gap'], integrality_tol=self.MIP_parameters['integrality_tol'])

                if bidder_specific_constraints is None:
                    logging.debug('SET ARGMAX ALLOCATION FOR ALL BIDDERS')
                    b = 0
                    for bidder in self.argmax_allocation[economy_key].keys():
                        self.argmax_allocation[economy_key][bidder][0] = X.x_star[b, :]
                        b = b + 1
                else:
                    logging.debug('SET ARGMAX ALLOCATION ONLY BIDDER SPECIFIC for {}'.format(list(bidder_specific_constraints.keys())[0]))
                    for bidder in bidder_specific_constraints.keys():
                        b = X.get_bidder_key_position(bidder_key=bidder)  # transform bidder_key into bidder position in MIP
                        self.argmax_allocation[economy_key][bidder][1] = X.x_star[b, :]  # now on position 1!

                for key,value in self.argmax_allocation[economy_key].items():
                    logging.debug(key + ':  %s | %s', value[0], value[1])

                self.elapsed_time_mip[economy_key].append(X.soltime)
                break
            except Exception:
                logging.warning('-----------------------------------------------')
                logging.warning('NOT SUCCESSFULLY SOLVED in attempt: %s \n', attempt)
                logging.warning(X.Mip.solve_details)
                if attempt == attempts:
                    X.Mip.export_as_lp(basename='UnsolvedMip_iter{}_{}'.format(self.mlca_iteration, economy_key),path=os.getcwd(), hide_user_names=False)
                    sys.exit('STOP, not solved succesfully in {} attempts\n'.format(attempt))
                
                # clear_session()
                logging.debug('REFITTING:')
                self.estimation_step(economy_key=economy_key)
            DNNs = OrderedDict(list((key, self.NN_models[economy_key][key].model) for key in list(self.NN_models[economy_key].keys())))
        del X
        del DNNs




    def tighten_bounds_IA(self, upper_bound_input, bidder_model, verbose=False):
        bidder = self.bidder_name
        Wb_total = util._clean_weights(util._get_model_weights(bidder_model))
        k = 0
        for j in range(len(util._get_model_layer_shapes(bidder_model, layer_type=['dense', 'input']))):  # loop over layers including input layer
                if j == 0:
                    self.upper_bounds_z[bidder][j] = np.array(upper_bound_input).reshape(-1, 1)
                    self.upper_bounds_s[bidder][j] = np.array(upper_bound_input).reshape(-1, 1)
                else:
                    W_plus = np.maximum(Wb_total[k].transpose(), 0)
                    W_minus = np.minimum(Wb_total[k].transpose(), 0)
                    self.upper_bounds_z[bidder][j] = np.ceil(np.maximum(W_plus @ self.upper_bounds_z[bidder][j-1] + Wb_total[k+1].reshape(-1, 1), 0)).astype(int)   # upper bound for z
                    self.upper_bounds_s[bidder][j] = np.ceil(np.maximum(-(W_minus @ self.upper_bounds_z[bidder][j-1] + Wb_total[k+1].reshape(-1, 1)), 0)).astype(int)  # upper bound  for s
                    k = k+2
        if verbose is True:
            logging.debug('Upper Bounds z:')
            for k, v in pd.DataFrame({k: pd.Series(l) for k, l in self.upper_bounds_z.items()}).fillna('-').items():
                logging.debug(v)
            logging.debug('\nUpper Bounds s:')
            for k, v in pd.DataFrame({k: pd.Series(l) for k, l in self.upper_bounds_s.items()}).fillna('-').items():
                logging.debug(v)

In [22]:
def _get_model_weights(model): # torch
    nnmodel = model.model
    weights = []
    for params in nnmodel.parameters():
        weights.append(params.detach().cpu().numpy().T)        
    return weights

def _get_model_layer_shapes(model, layer_type=None):
    ''' return layer output shapes instead, 
        if 'input' is given as desired layer type, insert input dim at the beginning.
        assumes torch model '''
    # nnmodel = self.Models[key]
    nnmodel = model.model
    Layer_shapes = []
    for i, (name, param) in enumerate(nnmodel.named_parameters()):
        if (i==0) and ('input' in layer_type): 
            Layer_shapes.append(param.shape[1])
        if any([x in name for x in layer_type]) and ('bias' in name):
            Layer_shapes.append(param.shape[0])
    return Layer_shapes

def _clean_weights(Wb):
    for v in range(0, len(Wb)-2, 2):
        Wb[v][abs(Wb[v]) <= 1e-8] = 0
        Wb[v+1][abs(Wb[v+1]) <= 1e-8] = 0
        zero_rows = np.where(np.logical_and((Wb[v] == 0).all(axis=0), Wb[v+1] == 0))[0]
        if len(zero_rows) > 0:
            logging.debug('Clean Weights (rows) %s', zero_rows)
            Wb[v] = np.delete(Wb[v], zero_rows, axis=1)
            Wb[v+1] = np.delete(Wb[v+1], zero_rows)
            Wb[v+2] = np.delete(Wb[v+2], zero_rows, axis=0)
    return(Wb)

bundles = np.random.randint(2, size=(5, 10))
values = np.random.randint(100, size=(5, 1))

In [23]:
#random bundles of 1 or 0 
bundles = np.random.randint(2, size=(5, 10))
values = np.random.randint(100, size=(5, 1))
#print(bundles)
#print(values)
model = MLCA_NN(X_train = bundles, Y_train=values)

X = MLCA_NNMIP(DNNs, L=self.MIP_parameters['bigM'])

model.initialize_model(model_parameters={'learning_rate': 0.001, 'architecture': [10, 10, 10], 'dropout': False, 'dropout_prob': 0.2, 'device': 'cpu', 'regularization_type': 'l2', 'regularization': 0.01})

model.fit(epochs=10, batch_size=32)

models = OrderedDict([(0, model)])

mip = MLCA_NNMIP(models, L = 6000)

mip.solve_mip()

NameError: name 'DNNs' is not defined

In [ ]:
def galo_nn_ilp(nn_model):

    layer = 1
    Wb = _clean_weights(_get_model_weights(nn_model))
    key = nn_model#do we need this?
    i=0 # do we need this
    m = Model(name='galo_nn')
    
    z = {}
    q = {}
    y = {}
    r = m.continuous_var(name='r')
    L = 6000
    

    
    m.add_constraint(values[i] - z[-1][0] <= r)
    m.add_constraint(z[-1][0] - values[i] <= r)
    for i in range(len(values)):    
        m.add_constraint(values[i] - z[(i, layer-1, 0)] + L * b[i] <= r)
        m.add_constraint(z[(i, layer<-1, 0)] - values[i] + L * (1- b[i]) <= r)
    

    for v in range(0, len(Wb), 2):
        W = Wb[v].transpose()
        b = Wb[v + 1]
        R, J = W.shape
        if v == 0:
            z.update({(i, 0, j): m.binary_var(name="x({})_{}".format(i, j)) for j in range(0, J)})
        
        z.update({(i, layer, r): m.continuous_var(lb=0, name="z({},{})_{}".format(i, layer, r)) for r in range(0, R)})
        #s in paper
        q.update({(i, layer, r): m.continuous_var(lb=0, name="s({},{})_{}".format(i, layer, r)) for r in range(0, R)}) # it is different from the initial code
        y.update({(i, layer, r): m.binary_var(name="y({},{})_{}".format(i, layer, r)) for r in range(0, R)}) # it is different from the initial code


        # if self.upper_bounds_z[key][layer][r][0] == 0 TODO
        # elif self.upper_bounds_s[key][layer][r][0] == 0: TODO
        # else:

        for r in range(0, R):#a lot of elif conditions here do it normally for now
            print(m.sum(W[r, j]*z[(i, layer-1, j)] for j in range(0, J)) + b[r] == z[(i, layer, r)] - q[(i, layer, r)])
            m.add_constraint(ct=(m.sum(W[r, j]*z[(i, layer-1, j)] for j in range(0, J)) + b[r] == z[(i, layer, r)] - q[(i, layer, r)]),
                                            ctname="AffineCT_Bidder{}_Layer{}_Row{}".format(i, layer, r))
            
            
            #m.add_constraint(ct=(q[(i, layer, r)] <= nn_model.upper_bounds_s[key][layer][r][0]*y[(i, layer, r)]),


            m.add_constraint(0<=z[(i, layer, r)])
            m.add_constraint(ct=z[(i, layer, r)] <= y[(i, layer, r)]*L, ctname="BinaryCT_Bidder{}_Layer{}_Row{}_Z".format(i, layer, r))


            m.add_constraint(0<=q[(i, layer, r)])
            m.add_constraint(ct=q[(i, layer, r)] <= (1-y[(i, layer, r)])*L, ctname="BinaryCT_Bidder{}_Layer{}_Row{}_S".format(i, layer, r))
        layer = layer + 1
    m.maximize(r)
    sol = m.solve()


model = MLCA_NN(X_train = bundles, Y_train=values)
model.initialize_model(model_parameters={'learning_rate': 0.001, 'architecture': [10, 10, 10], 'dropout': False, 'dropout_prob': 0.2, 'device': 'cpu', 'regularization_type': 'l2', 'regularization': 0.01})
model.fit(epochs=10, batch_size=32)
galo_nn_ilp(model)


KeyError: -1

In [ ]:
def _get_model_weights(model): # torch
    nnmodel = model.model
    weights = []
    for params in nnmodel.parameters():
        weights.append(params.detach().cpu().numpy().T)        
    return weights

def _get_model_layer_shapes(model, layer_type=None):
    ''' return layer output shapes instead, 
        if 'input' is given as desired layer type, insert input dim at the beginning.
        assumes torch model '''
    # nnmodel = self.Models[key]
    nnmodel = model.model
    Layer_shapes = []
    for i, (name, param) in enumerate(nnmodel.named_parameters()):
        if (i==0) and ('input' in layer_type): 
            Layer_shapes.append(param.shape[1])
        if any([x in name for x in layer_type]) and ('bias' in name):
            Layer_shapes.append(param.shape[0])
    return Layer_shapes

def _clean_weights(Wb):
    for v in range(0, len(Wb)-2, 2):
        Wb[v][abs(Wb[v]) <= 1e-8] = 0
        Wb[v+1][abs(Wb[v+1]) <= 1e-8] = 0
        zero_rows = np.where(np.logical_and((Wb[v] == 0).all(axis=0), Wb[v+1] == 0))[0]
        if len(zero_rows) > 0:
            logging.debug('Clean Weights (rows) %s', zero_rows)
            Wb[v] = np.delete(Wb[v], zero_rows, axis=1)
            Wb[v+1] = np.delete(Wb[v+1], zero_rows)
            Wb[v+2] = np.delete(Wb[v+2], zero_rows, axis=0)
    return(Wb)

bundles = np.random.randint(2, size=(5, 10))
values = np.random.randint(100, size=(5, 1))

In [35]:
count = 0
M = 10
#big integer
C = 6000
L = 6000
#while count < 10:
model = MLCA_NN(X_train = bundles, Y_train=values)
model.initialize_model(model_parameters={'learning_rate': 0.001, 'architecture': [10, 10, 10], 'dropout': False, 'dropout_prob': 0.2, 'device': 'cpu', 'regularization_type': 'l2', 'regularization': 0.01})
model.fit(epochs=10, batch_size=32)
weights = _get_model_weights(model)
layers = _get_model_layer_shapes(model, layer_type=['dense', 'input'])
Wb = _clean_weights(weights)
#print(Wb[0])
#print(layers)
#print(weights[5])    

[ 0.0100918  -0.00580223  0.01008647  0.01008843 -0.01007877 -0.010079
 -0.00973917  0.01008619  0.01000735  0.01009054]


In [ ]:
for i in range(len(values)):
        m  = Model(name='galo_nn')
        #create z^k variables
        z = {}
        y = {}
        q = {}
        #print(z)
        r = m.continuous_var(name='r')
        #x = m.binary_var_list(range(M), name='x')
        b = m.binary_var_list(range(len(values)), name='b')
        
        constraints = []
        constraints.append(values[i] - z[-1][0] <= r)
        constraints.append(z[-1][0] - values[i] <= r)
        
        for i in range(len(values)):
            constraints.append(values[i] - z[-1][0] + C * b[i] <= r)
            constraints.append(z[-1][0] - values[i] + C * (1- b[i]) <= r)
        

    count += 1
